<a href="https://colab.research.google.com/github/pavithrashaji/text-summarisation-research/blob/main/Graph_Based_Summarisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
text=pd.read_csv('/content/drive/MyDrive/DUC 2002 E/Input Files/DUC 2002 Extractive.csv')

In [ ]:
pretext=pd.read_csv('/content/drive/MyDrive/DUC 2002 E/Input Files/NSW_S.csv')

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import networkx as nx

#Form a dictionary of words
def preprocess(text):
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]
    words = [word for sentence in words for word in sentence]
    return words

import gensim.downloader as api

#Build the graph
def build_graph(words):
    model = api.load('word2vec-google-news-300')

    graph = nx.Graph()
    for i, word in enumerate(words):
        for j in range(i + 1, len(words)):
            if word != words[j]:
                if word in model.key_to_index and words[j] in model.key_to_index:
                    similarity = np.dot(model.get_vector(word), model.get_vector(words[j])) / (np.linalg.norm(model.get_vector(word)) * np.linalg.norm(model.get_vector(words[j])))
                    if similarity > 0:
                        graph.add_edge(word, words[j], weight=similarity)

    return graph

#Summarize Module
def summarize(text,pretext, num_sentences):
    words = preprocess(pretext)
    graph = build_graph(words)

    #Compute scores of each word
    scores = nx.pagerank(graph)

    #Rank the sentences in descending order
    ranked_sentences = sorted(((scores[word], sentence) for i, sentence in enumerate(sent_tokenize(text)) for word in word_tokenize(sentence.lower()) if word in scores), reverse=True)

    #Extract the required sentences
    summary=""
    count=0
    for score,sentence in ranked_sentences:
      if sentence not in summary and "---" not in sentence:
        summary=summary+" "+sentence
        count=count+1
      if count==num_sentences:
        break


    return summary


In [ ]:
print(summarize(text.iloc[0][2],pretext.iloc[0][2],num_sentences=15))

In [ ]:
for i in range(1,16):
  print("\nSummary:")
  print(i)
  print(summarize(text.iloc[i][2],pretext.iloc[i][2],num_sentences=15))
  print("\n\n")

In [ ]:
for i in range(16,31):
  print("\nSummary:")
  print(i)
  print(summarize(text.iloc[i][2],pretext.iloc[i][2],num_sentences=15))
  print("\n\n")

In [ ]:
for i in range(31,46):
  print("\nSummary:")
  print(i)
  print(summarize(text.iloc[i][2],pretext.iloc[i][2],num_sentences=15))
  print("\n\n")

In [ ]:
for i in range(46,59):
  print("\nSummary:")
  print(i)
  print(summarize(text.iloc[i][2],pretext.iloc[i][2],num_sentences=15))
  print("\n\n")
